# ライブラリ

In [ ]:
# ライブラリ
!pip install pulp
import pulp
import pandas as pd

# 2.1 連立一次方程式をPythonの数理最適化ライブラリで解く

In [ ]:
# 数理モデルを定義
problem = pulp.LpProblem("SLE", pulp.LpMaximize)

In [ ]:
# 変数を定義
x = pulp.LpVariable("x", cat = "Continuous")
y = pulp.LpVariable("y", cat = "Continuous")

In [ ]:
# 連立一次方程式を定義（数理モデルproblemに制約式を加える）
problem += 120 * x + 150 * y == 1440
problem += x + y == 10

# addConstraintでも可
#problem.addConstraint(120 * x + 150 * y == 1440)
#problem.addConstraint(x + y == 10)

In [ ]:
# 解く
status = problem.solve()

In [ ]:
# 結果
print(f"Status: {pulp.LpStatus[status]}")
print(f"x = {x.value()}, y = {y.value()}")

Status: Optimal
x = 2.0, y = 8.0


# 2.2 線形計画問題をPythonの数理最適化ライブラリで解く

In [ ]:
# 数理モデルを定義
problem = pulp.LpProblem("LP", pulp.LpMaximize)

In [ ]:
# 変数を定義
x = pulp.LpVariable("x", cat = "Continuous")
y = pulp.LpVariable("y", cat = "Continuous")

In [ ]:
# 線形計画問題を定義
problem += 1 * x + 3 * y <= 30
problem += 2 * x + 1 * y <= 40
problem += x >= 0
problem += y >= 0
problem += x + 2 * y

# 目的関数の追加は以下でも可
# problem.setObjective(x + 2 * y)

In [ ]:
# 解く
status = problem.solve()

In [ ]:
# 結果
print(f"Status: {pulp.LpStatus[status]}")
print(f"x = {x.value()}, y = {y.value()}\nobj = {problem.objective.value()}")

Status: Optimal
x = 18.0, y = 4.0
obj = 26.0


# 2.3 規模の大きな数理最適化問題をPythonの数理最適化ライブラリで解く

In [ ]:
# データのインポート
stock_df = pd.read_csv("https://raw.githubusercontent.com/ohmsha/PyOptBook/main/2.tutorial/stocks.csv")
require_df = pd.read_csv("https://raw.githubusercontent.com/ohmsha/PyOptBook/main/2.tutorial/requires.csv")
gain_df = pd.read_csv("https://raw.githubusercontent.com/ohmsha/PyOptBook/main/2.tutorial/gains.csv")

In [ ]:
# 製品のリスト
P = gain_df["p"].tolist()
P

['p1', 'p2', 'p3', 'p4']

In [ ]:
# 原料のリスト
M = stock_df["m"].tolist()
M

['m1', 'm2', 'm3']

In [ ]:
# 定数stock
stock = {row.m: row.stock for row in stock_df.itertuples()}
stock

{'m1': 35, 'm2': 22, 'm3': 27}

In [ ]:
# 定数require
require = {(row.p, row.m): row.require for row in require_df.itertuples()}
require

{('p1', 'm1'): 2,
 ('p1', 'm2'): 0,
 ('p1', 'm3'): 1,
 ('p2', 'm1'): 3,
 ('p2', 'm2'): 2,
 ('p2', 'm3'): 0,
 ('p3', 'm1'): 0,
 ('p3', 'm2'): 2,
 ('p3', 'm3'): 2,
 ('p4', 'm1'): 2,
 ('p4', 'm2'): 2,
 ('p4', 'm3'): 2}

In [ ]:
# 定数gain
gain = dict(zip(gain_df["p"], gain_df["gain"]))
gain

{'p1': 3, 'p2': 4, 'p3': 4, 'p4': 5}

In [ ]:
# 線形計画問題を定義
problem = pulp.LpProblem("LP2", pulp.LpMaximize)

In [ ]:
# 変数を定義
x = pulp.LpVariable.dicts("x", P, cat = "Continuous")

In [ ]:
# 制約式を定義

# 生産量は0以上
for p in P:
  problem += x[p] >= 0

# 生産は在庫の範囲で行う
for m in M:
  problem += pulp.lpSum([require[p, m] * x[p] for p in P]) <= stock[m]

In [ ]:
# 目的関数を定義
problem += pulp.lpSum([gain[p] * x[p] for p in P])

In [ ]:
# 解く
status = problem.solve()
print(f"Status: {pulp.LpStatus[status]}")

Status: Optimal


In [ ]:
# 解と目的関数値
for p in P:
  print(f"{p}: {x[p].value()}")
print(f"obj = {problem.objective.value()}")

p1: 12.142857
p2: 3.5714286
p3: 7.4285714
p4: 0.0
obj = 80.42857099999999


## 整数計画問題として解く

In [ ]:
# 整数計画問題として数理モデルを定義
problem = pulp.LpProblem("IP", pulp.LpMaximize)

# 変数の定義を修正
x = pulp.LpVariable.dicts("x", P, cat = "Integer")

# 制約式を追加
for p in P:
  problem += x[p] >= 0
for m in M:
  problem += pulp.lpSum([require[p, m] * x[p] for p in P]) <= stock[m]

# 目的関数を定義
problem += pulp.lpSum([gain[p] * x[p] for p in P])

In [ ]:
# 解きなおし
status = problem.solve()

In [ ]:
# 結果
print(f"Status: {pulp.LpStatus[status]}")
for p in P:
  print(f"{p}: {x[p].value()}")
print(f"obj = {problem.objective.value()}")

Status: Optimal
p1: 13.0
p2: 3.0
p3: 7.0
p4: -0.0
obj = 79.0
